**Performance Metrics**



In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

import tensorflow as tf
from tensorflow.keras import backend as K

def recall(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    recall_value = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall_value

def precision(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision_value = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision_value

def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    f1_value = 2 * (prec * rec) / (prec + rec + tf.keras.backend.epsilon())
    return f1_value

def jaccard_index(y_true, y_pred, threshold=0.5):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    y_true_f = tf.reshape(y_true, (-1,))
    y_pred_f = tf.reshape(y_pred, (-1,))
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    sum_ = tf.reduce_sum(y_true_f + y_pred_f)
    jaccard_value = intersection / (sum_ - intersection + tf.keras.backend.epsilon())
    return jaccard_value

def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def combined_loss(y_true, y_pred):
    dice = dice_loss(y_true, y_pred)
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return dice + bce

**Here in our AI vision system architecture:**

**Part One:**

**In preprocessing:The dataset included annotations, which I used to generate the masks. I also focused on resizing both the image and masks to** **256x256, which is a very suitable input size because, in the panoramic x-ray, the target zone is centered, so we don't need the rest** **of the image.** **This is because when I initially set the input size to 512*512, this made the model focus more on the pixels in** **the background. I applied data augmentation using techniques tailored to the nature of the dental x-ray input. I used a** **data generator, which gave me full control over the data during training, using a batch-by-batch approach.** **I also applied normalization and set a threshold for the masks, which is what the model needed for the segmentation output.**

**Part Two:**

**Regarding the Custom Segmentation U-Net with VGG19 as the backbone, I replaced the encoder with VGG19, a type of transfer learning** **model pre-trained on the ImageNet dataset, to leverage the power of feature extraction, reduce training time, and achieve higher accuracy.**
**I also developed the decoder with improvements, such as Batch Normalization and Dropout, to enhance model stability and prevent overfitting.**

**Part Three:**

**Finally, the model produces a mask that defines the problem regions.**

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# U-Net Model with VGG19 Backbone
def unet_vgg19_model(input_shape):
    # Load VGG19 as the encoder (backbone)
    vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze VGG19 layers initially
    for layer in vgg_base.layers:
        layer.trainable = False

    # Encoder: Use VGG19 layers directly (after pooling)
    c1 = vgg_base.get_layer('block1_conv2').output  # 256x256x64
    p1 = vgg_base.get_layer('block1_pool').output   # 128x128x64
    c2 = vgg_base.get_layer('block2_conv2').output  # 128x128x128
    p2 = vgg_base.get_layer('block2_pool').output   # 64x64x128
    c3 = vgg_base.get_layer('block3_conv4').output  # 64x64x256
    p3 = vgg_base.get_layer('block3_pool').output   # 32x32x256
    c4 = vgg_base.get_layer('block4_conv4').output  # 32x32x512
    p4 = vgg_base.get_layer('block4_pool').output   # 16x16x512
    c5 = vgg_base.get_layer('block5_conv4').output  # 16x16x512

    # Bridge
    b1 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)  # 16x16x1024
    b1 = BatchNormalization()(b1)
    b1 = Conv2D(1024, (3, 3), activation='relu', padding='same')(b1)  # 16x16x1024
    b1 = BatchNormalization()(b1)
    b1 = Dropout(0.5)(b1)

    # Decoder
    # Up-sample to 32x32
    u5 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(b1)  # 32x32x512
    u5 = concatenate([u5, c4])  # 32x32x1024
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(u5)  # 32x32x512
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(c5)  # 32x32x512
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.5)(c5)

    # Up-sample to 64x64
    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)  # 64x64x256
    u6 = concatenate([u6, c3])  # 64x64x512
    c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(u6)  # 64x64x256
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(c6)  # 64x64x256
    c6 = BatchNormalization()(c6)
    c6 = Dropout(0.3)(c6)

    # Up-sample to 128x128
    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)  # 128x128x128
    u7 = concatenate([u7, c2])  # 128x128x256
    c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(u7)  # 128x128x128
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(c7)  # 128x128x128
    c7 = BatchNormalization()(c7)
    c7 = Dropout(0.3)(c7)

    # Up-sample to 256x256
    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)  # 256x256x64
    u8 = concatenate([u8, c1])  # 256x256x128
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(u8)  # 256x256x64
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(c8)  # 256x256x64
    c8 = BatchNormalization()(c8)

    # Output layer for binary segmentation
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c8)  # 256x256x1

    model = Model(inputs=vgg_base.input, outputs=outputs)
    return model

# Build and compile the model
input_shape = (256, 256, 3)
batch_size = 8

In [ ]:
model = unet_vgg19_model(input_shape)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=combined_loss,
              metrics=[dice_coefficient,recall,precision,jaccard_index,f1_score])

# Define callbacks
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_dice_coefficient', mode='max')
early_stopping = EarlyStopping(monitor='val_dice_coefficient', patience=2, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_dice_coefficient', factor=0.2, patience=3, min_lr=1e-6)

I0000 00:00:1753033186.039541      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 256, 256,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 128, 128,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 128, 128,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 128, 128,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 64, 64,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 64, 64,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 64, 64,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 64, 64,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv4        │ (None, 64, 64,    │    590,080 │ block3_conv3[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 32, 32,    │          0 │ block3_conv4[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 32, 32,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 32, 32,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 32, 32,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv4        │ (None, 32, 32,    │  2,359,808 │ block4_conv3[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 16, 16,    │          0 │ block4_conv4[0][

 Total params: 46,386,625 (176.95 MB)

 Trainable params: 26,354,305 (100.53 MB)

 Non-trainable params: 20,032,320 (76.42 MB)

In [ ]:
# Train the model for more epochs
history = model.fit(
    train_data_gen,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=valid_data_gen,
    validation_steps=valid_steps_per_epoch,
    epochs=10,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

Epoch 1/10


I0000 00:00:1753033271.724590    1123 service.cc:148] XLA service 0x7e63380063d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753033271.725294    1123 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753033273.072374    1123 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753033298.781463    1123 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


595/597 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - dice_coefficient: 0.0907 - f1_score: 0.3131 - jaccard_index: 0.1997 - loss: 1.4649 - precision: 0.2300 - recall: 0.7041

E0000 00:00:1753033454.888166    1124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753033455.145099    1124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753033457.998448    1124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753033458.254690    1124 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


597/597 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - dice_coefficient: 0.0908 - f1_score: 0.3135 - jaccard_index: 0.2000 - loss: 1.4644 - precision: 0.2304 - recall: 0.7041

E0000 00:00:1753033540.670475    1126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753033540.904852    1126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753033541.524577    1126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1753033541.782055    1126 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


597/597 ━━━━━━━━━━━━━━━━━━━━ 289s 418ms/step - dice_coefficient: 0.0908 - f1_score: 0.3137 - jaccard_index: 0.2002 - loss: 1.4641 - precision: 0.2306 - recall: 0.7041 - val_dice_coefficient: 0.2217 - val_f1_score: 0.6358 - val_jaccard_index: 0.4697 - val_loss: 0.9328 - val_precision: 0.6341 - val_recall: 0.6484 - learning_rate: 1.0000e-04
Epoch 2/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 155s 261ms/step - dice_coefficient: 0.1977 - f1_score: 0.5794 - jaccard_index: 0.4126 - loss: 1.0382 - precision: 0.5077 - recall: 0.7101 - val_dice_coefficient: 0.5222 - val_f1_score: 0.6579 - val_jaccard_index: 0.4932 - val_loss: 0.6576 - val_precision: 0.5851 - val_recall: 0.7637 - learning_rate: 1.0000e-04
Epoch 3/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 156s 261ms/step - dice_coefficient: 0.4157 - f1_score: 0.6393 - jaccard_index: 0.4756 - loss: 0.7123 - precision: 0.6154 - recall: 0.6907 - val_dice_coefficient: 0.6329 - val_f1_score: 0.6886 - val_jaccard_index: 0.5297 - val_loss: 0.5375 - val_precision: 0.6939 - va